In [1]:
import numpy as np
from scipy import stats

# Mean reciprocal rank (MRR)

\begin{align}
MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{rank_i}
\end{align}
Where: <br>
|Q| is the number of queries or user lists <br>
rank_i is the position of the first relevant item for the i-th query

GT:
- The more the relevant item will be in the top ranked items, the more MRR value will be.

In [2]:
def mean_reciprocal_rank(relevance_lists):
    """
    Calculate Mean Reciprocal Rank (MRR)
    
    Args:
    relevance_lists (list of lists): Each inner list represents relevance scores for a query,
                                     where 1 indicates a relevant item and 0 an irrelevant item.
    
    Returns:
    float: The Mean Reciprocal Rank
    """
    
    reciprocal_ranks = []
    for relevance in relevance_lists:
        try:
            # Find the index of the first relevant item (1)
            rank = relevance.index(1) + 1  # +1 because index is 0-based
            reciprocal_ranks.append(1 / rank)
        except ValueError:
            # If no relevant item is found, use 0
            reciprocal_ranks.append(0)
    
    return np.mean(reciprocal_ranks)

# Example usage
relevance_lists = [
    [0, 0, 1, 0, 0],  # First relevant item at position 3
    [1, 0, 0, 0, 0],  # First relevant item at position 1
    [0, 0, 0, 0, 0],  # No relevant item
    [0, 1, 0, 0, 0]   # First relevant item at position 2
]

mrr = mean_reciprocal_rank(relevance_lists)
print(f"Mean Reciprocal Rank: {mrr}")

Mean Reciprocal Rank: 0.4583333333333333


# Mean Average Precision (MAP)

\begin{align}
MAP = \frac{1}{|Q|} \sum_{q=1}^{|Q|} AP(q)
\end{align}
Where:
|Q| is the number of queries
AP(q) is the Average Precision for a single query q
The Average Precision (AP) for a single query is calculated as:
\begin{align}
AP = \sum_{k=1}^n P(k) \times rel(k)
\end{align}
Where:
k is the rank in the sequence of retrieved items
n is the number of retrieved items
P(k) is the precision at cut-off k in the list
rel(k) is an indicator function equaling 1 if the item at rank k is relevant, zero otherwise

GT:
- The higher the relevance score of the top ranked items are, the more MAP value will be.

In [3]:
def average_precision(y_true, y_scores):
    """Calculate average precision for a single query"""
    sorted_indices = np.argsort(y_scores)[::-1]
    y_true = np.take(y_true, sorted_indices)
    
    precisions = np.cumsum(y_true) / (np.arange(len(y_true)) + 1)
    print(f'sorted_indices: {sorted_indices}, y_true: {y_true}, {np.cumsum(y_true)}, {np.arange(len(y_true)) + 1}, {precisions.round(2)}')
    return np.sum(precisions * y_true) / np.sum(y_true)

def mean_average_precision(y_true, y_scores):
    """Calculate Mean Average Precision for multiple queries"""
    aps = [average_precision(y_true[i], y_scores[i]) for i in range(len(y_true))]
    return np.mean(aps)

# Example usage
y_true = [
    [1, 0, 1, 1, 0],  # Relevance for query 1
    [0, 1, 1, 0, 0]   # Relevance for query 2
]
y_scores = [
    [0.9, 0.2, 0.7, 0.8, 0.1],  # Prediction scores for query 1
    [0.1, 0.8, 0.9, 0.3, 0.8]   # Prediction scores for query 2
]

map_score = mean_average_precision(y_true, y_scores)
print(f"Mean Average Precision: {map_score}")

sorted_indices: [0 3 2 1 4], y_true: [1 1 1 0 0], [1 2 3 3 3], [1 2 3 4 5], [1.   1.   1.   0.75 0.6 ]
sorted_indices: [2 4 1 3 0], y_true: [1 0 1 0 0], [1 1 2 2 2], [1 2 3 4 5], [1.   0.5  0.67 0.5  0.4 ]
Mean Average Precision: 0.9166666666666666


# Normalized Discounted Cumulative Gain (NDCG)

Normalized Discounted Cumulative Gain (NDCG) measures how well a ranking algorithm orders items based on their relevance, with a focus on the top-ranked items.

\begin{align}
NDCG_p = \frac{DCG_p}{IDCG_p}
\end{align}
Where:
\begin{align}
DCG_p = \sum_{i=1}^p \frac{2^{rel_i} - 1}{\log_2(i + 1)}
\end{align}
And:
\begin{align}
IDCG_p = \sum_{i=1}^{|REL_p|} \frac{2^{rel_i} - 1}{\log_2(i + 1)}
\end{align}
In these formulas:
$p$ is the position up to which NDCG is calculated
$rel_i$ is the relevance score of the item at position $i$
$|REL_p|$ is the list of relevant documents (ordered by their relevance) up to position $p$

GT:
- The higher the relevance score of the top ranked items are, the more DCG value will be.

In [4]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    print(f'r: {r}, {np.power(2, r)}, {np.subtract(np.power(2, r), 1)}, {np.log2(np.arange(2, r.size + 2)).round(2)}, \
        {(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2))).round(2)}, \
        {np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2))).round(2)}')
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# Example usage
# relevance_scores = [3, 2, 3, 0, 1, 2] # 0.8755
# relevance_scores = [4, 2, 3, 0, 1, 2] # 0.9196
relevance_scores = [2, 2, 3, 0, 1, 2] # 0.7272
k = 5
ndcg = ndcg_at_k(relevance_scores, k)
print(f"NDCG@{k}: {ndcg}")

r: [3. 2. 2. 2. 1.], [8. 4. 4. 4. 2.], [7. 3. 3. 3. 1.], [1.   1.58 2.   2.32 2.58],         [7.   1.89 1.5  1.29 0.39],         12.07
r: [2. 2. 3. 0. 1.], [4. 4. 8. 1. 2.], [3. 3. 7. 0. 1.], [1.   1.58 2.   2.32 2.58],         [3.   1.89 3.5  0.   0.39],         8.78
NDCG@5: 0.7272929761069984


In [5]:
'''
In the Python implementation of NDCG above, we used relevance_scores for evaluation.
Let's say that this relevance_scores came from a method we want to evaluate.
How can we use ground truth score for evaluation using NDCG?
'''
def ndcg_at_k(ground_truth, predicted_order, k):
    # Sort ground truth scores based on predicted order
    reordered_ground_truth = [ground_truth[i] for i in predicted_order]
    
    # Calculate DCG of the reordered ground truth
    dcg = dcg_at_k(reordered_ground_truth, k)
    
    # Calculate IDCG (using original ground truth, sorted in descending order)
    idcg = dcg_at_k(sorted(ground_truth, reverse=True), k)
    
    if idcg == 0:
        return 0.
    
    return dcg / idcg

# Example usage
ground_truth_scores = [3, 2, 3, 0, 1, 2]  # Ground truth relevance scores
predicted_order = [0, 2, 1, 5, 3, 4]  # Predicted ranking (as indices of ground_truth_scores)
k = 5

ndcg = ndcg_at_k(ground_truth_scores, predicted_order, k)
print(f"NDCG@{k}: {ndcg}")

r: [3. 3. 2. 2. 0.], [8. 8. 4. 4. 1.], [7. 7. 3. 3. 0.], [1.   1.58 2.   2.32 2.58],         [7.   4.42 1.5  1.29 0.  ],         14.21
r: [3. 3. 2. 2. 1.], [8. 8. 4. 4. 2.], [7. 7. 3. 3. 1.], [1.   1.58 2.   2.32 2.58],         [7.   4.42 1.5  1.29 0.39],         14.6
NDCG@5: 0.973494864667227


# Rank correlation 

Rank correlation is a measure of the relationship between the rankings of two variables.
It assesses how well the relationship between two variables can be described using a monotonic function, without making any assumptions about the frequency distribution of the variables.
Two of the most common rank correlation coefficients are:
1. Spearman's rank correlation coefficient (ρ or rs)
2. Kendall's tau (τ)

The formula for **Spearman's rank correlation coefficient** is:
\begin{align}
\rho = 1 - \frac{6 \sum d_i^2}{n(n^2 - 1)}
\end{align}

In [6]:
def spearman_rank_correlation(x, y):
    # Convert to numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Get the ranks
    x_ranks = stats.rankdata(x)
    y_ranks = stats.rankdata(y)
    
    # Calculate the difference in ranks
    d = x_ranks - y_ranks
    print(f'y_ranks: {y_ranks}, {d}')
    
    # Calculate n
    n = len(x)
    
    # Calculate rho
    rho = 1 - (6 * np.sum(d**2)) / (n * (n**2 - 1))
    
    return rho

# Example usage
x = [1, 2, 3, 4, 5]
y = [2, 1, 2, 4, 5]

correlation = spearman_rank_correlation(x, y)
print(f"Spearman's rank correlation coefficient: {correlation}")

# Verify with scipy's implementation
scipy_correlation, pvalue = stats.spearmanr(x, y)
print(f"SciPy's Spearman correlation: {scipy_correlation}, pvalue: {pvalue}")

y_ranks: [2.5 1.  2.5 4.  5. ], [-1.5  1.   0.5  0.   0. ]
Spearman's rank correlation coefficient: 0.825
SciPy's Spearman correlation: 0.8207826816681233, pvalue: 0.08858700531354381
